In [1]:
import sys 
sys.path.append('/Users/justinvhuang/Desktop/CSE-6242-Group-Project')

In [37]:
import pandas as pd

In [38]:
df = pd.read_json("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/fin_anime_dfv1.json")

In [40]:
df.columns

Index(['imdb_title_title', 'imdb_title_titleId', 'anime_Rating',
       'anime_English name', 'anime_Genres', 'anime_Other name', 'anime_Score',
       'anime_Synopsis', 'anime_anime_id', 'imdb_name_basics_primaryName',
       'plot', 'keywords', 'image', 'Producers', 'Licensors', 'Studios',
       'Image URL', 'Episodes', 'Genres', 'Source', 'Favorites', 'Aired',
       'Members', 'Duration', 'text', 'Name', 'anime_id', 'popular_recs',
       'cf_recs', 'tokens'],
      dtype='object')

In [2]:
from enum import Enum

import torch
from pydantic import BaseModel, constr

import outlines

import pandas as pd
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm
from utils.textpreprocessing import TextPreprocessor

In [3]:
textprepo = TextPreprocessor()

In [4]:
model = outlines.models.llamacpp("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/prompting/mistral-7b-instruct-v0.2-gguf/mistral-7b-instruct-v0.2.Q5_K_M.gguf")

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/justinvhuang/Desktop/CSE-6242-Group-Project/prompting/mistral-7b-instruct-v0.2-gguf/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.

In [5]:
encode_kwargs = {"normalize_embeddings": True}
embedding_function = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={"device": "cpu"},
    encode_kwargs=encode_kwargs,
)

In [7]:
new_db = FAISS.load_local("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/vector_database_creation/faiss_anime_index_v2", embedding_function,allow_dangerous_deserialization=True)

In [19]:
def are_words_in_string(word_list, input_string):
    for word in word_list:
        if word in input_string:
            return True
    return False

In [27]:
query = "treasure"
query_token = textprepo.preprocess_text(query)

def filter_fn(metadata):
    return any(word in query for word in list(query_token)) or metadata["score"] > 5.0


results = new_db.similarity_search(query, filter=filter_fn, k=10)
results

[Document(page_content="what follows is a mad dash by not only the treasure hunters but also the inhabitants of koumi to secure anne and mary's long lost booty. all the while, however, conan, kogorou, and the police search for the one thing far greater than riches—justice.", metadata={'anime_id': 2171, 'cast': 'UNKNOWN', 'episodes': '1.0', 'genre': 'Adventure, Comedy, Mystery', 'source': 'Manga', 'Duration': '1 hr 45 min', 'name': 'Detective Conan Movie 11: Jolly Roger in the Deep Azure', 'tokens': 'treasure pay trip chaos detective dare mad second conan send mystery big thing reef mouri robbery manga regard boy harsh azure acquaint famous hill encounter long reason booty daughter group \n\n discover cutlass far edogawa read sonoko roger fever anne scenic search follow movie hunt justice entertainment museum visit tourist plunge small isle mix murder jolly secure wonder join legend mary kogorou arrive lose scour friend recent clue pistol 11 water tourism deep rich bang luscious coral r

In [35]:
context = list(set([results[x].page_content for x in range(len(results))]))

In [36]:
anime_id_list = set([results[x].metadata['anime_id'] for x in range(len(results))])

In [5]:

prompt = """You are a sentiment-labelling assistant.
Is the following review positive or negative?

Review: This restaurant is just awesome!
"""

generator = outlines.generate.choice(model, ["Positive", "Negative"])
answer = generator(prompt)


llama_print_timings:        load time =    9816.53 ms
llama_print_timings:      sample time =       0.48 ms /     4 runs   (    0.12 ms per token,  8333.33 tokens per second)
llama_print_timings: prompt eval time =    9816.43 ms /    30 tokens (  327.21 ms per token,     3.06 tokens per second)
llama_print_timings:        eval time =     970.24 ms /     3 runs   (  323.41 ms per token,     3.09 tokens per second)
llama_print_timings:       total time =   10809.93 ms /    33 tokens


In [6]:
answer

'Negative'

In [7]:
class Weapon(str, Enum):
    sword = "sword"
    axe = "axe"
    mace = "mace"
    spear = "spear"
    bow = "bow"
    crossbow = "crossbow"


class Armor(str, Enum):
    leather = "leather"
    chainmail = "chainmail"
    plate = "plate"


class Character(BaseModel):
    name: constr(max_length=10)
    age: int
    armor: Armor
    weapon: Weapon
    strength: int

In [9]:
# Construct structured sequence generator
generator = outlines.generate.json(model, Character)

# Draw a sample
rng = torch.Generator(device="cpu")
rng.manual_seed(789001)

character = generator("Give me a character description", rng=rng)

print(repr(character))
# Character(name='Anderson', age=28, armor=<Armor.chainmail: 'chainmail'>, weapon=<Weapon.sword: 'sword'>, strength=8)

character = generator("Give me an interesting character description", rng=rng)

print(repr(character))


llama_print_timings:        load time =    9816.53 ms
llama_print_timings:      sample time =       5.60 ms /    50 runs   (    0.11 ms per token,  8922.20 tokens per second)
llama_print_timings: prompt eval time =    1910.41 ms /     6 tokens (  318.40 ms per token,     3.14 tokens per second)
llama_print_timings:        eval time =   16059.73 ms /    49 runs   (  327.75 ms per token,     3.05 tokens per second)
llama_print_timings:       total time =   18108.65 ms /    55 tokens


Character(name='Aria Stark', age=25, armor=<Armor.plate: 'plate'>, weapon=<Weapon.crossbow: 'crossbow'>, strength=90)



llama_print_timings:        load time =    9816.53 ms
llama_print_timings:      sample time =       6.19 ms /    55 runs   (    0.11 ms per token,  8888.17 tokens per second)
llama_print_timings: prompt eval time =    2244.06 ms /     7 tokens (  320.58 ms per token,     3.12 tokens per second)
llama_print_timings:        eval time =   17710.00 ms /    54 runs   (  327.96 ms per token,     3.05 tokens per second)
llama_print_timings:       total time =   20101.47 ms /    61 tokens


Character(name="Professor'", age=53, armor=<Armor.leather: 'leather'>, weapon=<Weapon.crossbow: 'crossbow'>, strength=120)


In [10]:
examples = [
    ("The food was disgusting", "Negative"),
    ("We had a fantastic night", "Positive"),
    ("Recommended", "Positive"),
    ("The waiter was rude", "Negative")
]

@outlines.prompt
def labelling(to_label, examples):
    """You are a sentiment-labelling assistant.

    {% for example in examples %}
    {{ example[0] }} // {{ example[1] }}
    {% endfor %}
    {{ to_label }} //
    """

prompt = labelling("Just awesome", examples)
answer = outlines.generate.text(model)(prompt, max_tokens=100)


llama_print_timings:        load time =    9816.53 ms
llama_print_timings:      sample time =      12.81 ms /   100 runs   (    0.13 ms per token,  7807.62 tokens per second)
llama_print_timings: prompt eval time =    6002.63 ms /    49 tokens (  122.50 ms per token,     8.16 tokens per second)
llama_print_timings:        eval time =   32582.94 ms /    99 runs   (  329.12 ms per token,     3.04 tokens per second)
llama_print_timings:       total time =   38843.40 ms /   148 tokens


In [11]:
answer

" Positive\n\nThese sentences have clear positive or negative sentiments, and it's easy for me to determine the label based on the context. However, sometimes the sentiment can be more nuanced, and it may require a deeper understanding of the context to accurately label it. In such cases, additional information can be helpful to make an informed decision. For example:\n\nThe dish looked beautiful but didn't taste good // Neutral (visually appealing but not satisfying taste-wise"

In [12]:
@outlines.prompt
def build_reAct_prompt(question):
    """What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
    Tho 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado ...
    Act 2: Search 'Colorado orogeny'
    Obs 2: The Colorado orogeny was an episode of mountain building (an orogeny) ...
    Tho 3: It does not mention the eastern sector. So I need to look up eastern sector.
    ...
    Tho 4: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
    Act 5: Finish '1,800 to 7,000 ft'
    {{ question }}
    """


@outlines.prompt
def add_mode(i, mode, result, prompt):
    """{{ prompt }}
    {{ mode }} {{ i }}: {{ result }}
    """


def search_wikipedia(query: str):
    url = f"https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&titles={query}&origin=*"
    response = requests.get(url)
    page = response.json()["query"]["pages"]
    return ".".join(list(page.values())[0]["extract"].split(".")[:2])

In [13]:
prompt = build_reAct_prompt("Where is Apple Computers headquarted? ")
complete = outlines.generate.text(model)

In [15]:
for i in range(1, 10):
    mode = complete.generate_choice(prompt, choices=["Tho", "Act"], max_tokens=128)
    prompt = add_mode(i, mode, "", prompt)

    if mode == "Tho":
        thought = complete(prompt, stop_at="\n", max_tokens=128)
        prompt += f"{thought}"
    elif mode == "Act":
        action = complete.generate_choice(
            prompt, choices=["Search", "Finish"], max_tokens=128
        )
        prompt += f"{action} '"

        subject = complete(
            prompt, stop_at=["'"], max_tokens=128
        )  # Apple Computers headquartered
        subject = " ".join(subject.split()[:2])
        prompt += f"{subject}'"

        if action == "Search":
            result = search_wikipedia(subject)
            prompt = add_mode(i, "Obs", result, prompt)
        else:
            break

AttributeError: 'LlamaSequenceGenerator' object has no attribute 'generate_choice'

In [17]:
@outlines.prompt
def take_order(order):
    """You are the owner of a pizza parlor. Customers \
    send you orders from which you need to extract:

    1. The pizza that is ordered
    2. The number of pizzas

    # EXAMPLE

    ORDER: I would like one Margherita pizza
    RESULT: {"pizza": "Margherita", "number": 1}

    # OUTPUT INSTRUCTIONS

    Answer in valid JSON. Here are the different objects relevant for the output:

    Order:
        pizza (str): name of the pizza
        number (int): number of pizzas

    Return a valid JSON of type "Order"

    # OUTPUT

    ORDER: {{ order }}
    RESULT: """

In [18]:
from enum import Enum
from pydantic import BaseModel

class Pizza(str, Enum):
    margherita = "Margherita"
    pepperonni = "Pepperoni"
    calzone = "Calzone"

class Order(BaseModel):
    pizza: Pizza
    number: int

In [19]:
orders = [
    "Hi! I would like to order two pepperonni pizzas and would like them in 30mins.",
    "Is it possible to get 12 margheritas?"
]
prompts = [take_order(order) for order in orders]

generator = outlines.generate.json(model, Order)

results = generator(prompts)
print(results)


llama_print_timings:        load time =    9816.53 ms
llama_print_timings:      sample time =       1.62 ms /    17 runs   (    0.10 ms per token, 10474.43 tokens per second)
llama_print_timings: prompt eval time =   16551.35 ms /   180 tokens (   91.95 ms per token,    10.88 tokens per second)
llama_print_timings:        eval time =    5247.16 ms /    16 runs   (  327.95 ms per token,     3.05 tokens per second)
llama_print_timings:       total time =   21842.41 ms /   196 tokens

llama_print_timings:        load time =    9816.53 ms
llama_print_timings:      sample time =       1.75 ms /    18 runs   (    0.10 ms per token, 10262.26 tokens per second)
llama_print_timings: prompt eval time =   15540.05 ms /   168 tokens (   92.50 ms per token,    10.81 tokens per second)
llama_print_timings:        eval time =    5773.70 ms /    17 runs   (  339.63 ms per token,     2.94 tokens per second)
llama_print_timings:       total time =   21360.72 ms /   185 tokens


[Order(pizza=<Pizza.pepperonni: 'Pepperoni'>, number=2), Order(pizza=<Pizza.margherita: 'Margherita'>, number=12)]


In [20]:
@outlines.prompt
def customer_support(request):
    """You are an experienced customer success manager.

    Given a request from a client, you need to determine when the
    request is urgent using the label "URGENT" or when it can wait
    a little with the label "STANDARD".

    # Examples

    Request: "How are you?"
    Label: STANDARD

    Request: "I need this fixed immediately!"
    Label: URGENT

    # TASK

    Request: {{ request }}
    Label: """

In [21]:
from outlines.generate.samplers import greedy

generator = outlines.generate.choice(model, ["URGENT", "STANDARD"], sampler=greedy)

ModuleNotFoundError: No module named 'outlines.generate.samplers'

In [23]:
from outlines.generate import greedy

ImportError: cannot import name 'greedy' from 'outlines.generate' (/Users/justinvhuang/miniconda3/envs/langchain/lib/python3.10/site-packages/outlines/generate/__init__.py)